# This note book load date from h5 file and save them to csv file

TODO :
- use Spark to have more data

In [1]:
import numpy as np
import pandas as pd

import os
import glob
import time
import csv

import h5py
import hdf5_getters
from hdf5_getters import *

from operator import methodcaller
%load_ext autoreload

In [3]:
def count_all_files(base_directory, ext='.h5') :
    cnt = 0
    for root, dirs, files in os.walk(base_directory):
        files = glob.glob(os.path.join(root,'*'+ext))
        cnt += len(files)
    return cnt

In [4]:
def get_all(getter, base_directory, ext='.h5'):
    values = []
    for root, dirs, files in os.walk(base_directory):
        files = glob.glob(os.path.join(root,'*'+ext))
        for f in files:
            h5 = hdf5_getters.open_h5_file_read(f)
            values.append( getter(h5) )
            h5.close()
    return values

In [5]:
def get_file_fields(file, all_getters):
    fields = []
    for getter_name in all_getters:
        getter = getattr(hdf5_getters, getter_name)
        fields.append(getter(file))
    return fields

In [2]:
base_directory = './millionsongsubset_full/MillionSongSubset/data'
import getpass
if getpass.getuser() == "benjamin":
    base_directory = "/home/benjamin/" + base_directory
base_directory

'/home/benjamin/./millionsongsubset_full/MillionSongSubset/data'

In [7]:
all_getters = list(filter(lambda x: x[:3] == 'get',hdf5_getters.__dict__.keys()))
print(all_getters)
print('Number of getters:', len(all_getters))

['get_mode', 'get_bars_confidence', 'get_artist_familiarity', 'get_year', 'get_analysis_sample_rate', 'get_beats_start', 'get_artist_latitude', 'get_artist_id', 'get_segments_confidence', 'get_bars_start', 'get_track_7digitalid', 'get_artist_name', 'get_similar_artists', 'get_key_confidence', 'get_danceability', 'get_tempo', 'get_segments_timbre', 'get_song_id', 'get_artist_mbid', 'get_end_of_fade_in', 'get_time_signature_confidence', 'get_artist_playmeid', 'get_segments_start', 'get_loudness', 'get_artist_terms_freq', 'get_tatums_start', 'get_time_signature', 'get_artist_location', 'get_tatums_confidence', 'get_beats_confidence', 'get_audio_md5', 'get_title', 'get_artist_7digitalid', 'get_song_hotttnesss', 'get_artist_hotttnesss', 'get_artist_mbtags', 'get_release_7digitalid', 'get_num_songs', 'get_artist_mbtags_count', 'get_release', 'get_artist_terms_weight', 'get_segments_loudness_max', 'get_artist_longitude', 'get_duration', 'get_segments_pitches', 'get_artist_terms', 'get_mode_co

#### Building the dataframe (only subset)



In [ ]:
#iterate over all the .h5 files, starting from the base directory
#note: takes between ~8 to ~10 minutes
start_time = time.time()

values = []
ext = '.h5'
i = 0
for root, dirs, files in os.walk(base_directory):
    files = glob.glob(os.path.join(root, '*'+ext))
    for f in files:
        h5 = hdf5_getters.open_h5_file_read(f)
        values.append(get_file_fields(h5, all_getters))
        h5.close()
        if(i%100==0):
            print(i*1.0/nbr_files*100,"% chargée")
        i = i+1

print("--- Execution time: %s seconds ---" % (time.time() - start_time))

In [9]:
column_names = [name[4:].replace('_', ' ') for name in all_getters]

In [ ]:
#create a dataframe with the given column names
data = pd.DataFrame(columns=column_names)
data.head()

In [ ]:
#fill the dataframe row by row
start_time = time.time()
nb_entries = len(values)
for i in range(0, nb_entries):
    if(i%100==0):
            print(i*1.0/nb_entries*100,"% chargée")
    data.loc[i] = values[i]
print("--- Execution time: %s seconds ---" % (time.time() - start_time))

In [10]:
def save_to_csv(data, filename, folder='./data/'):
    #note: newline='' prevents Python 3 from adding a new line after writing each row
    with open(folder+filename, 'w', newline='') as f:
        writer = csv.writer(f)
        
        if(type(data[0]) == np.ndarray):
            #unfold the pandas series into an array of array
            array = [subarray for subarray in data]
            writer.writerows(array)
        else:
            writer.writerow(data)

#### Writing the dataframe to .csv

In [11]:
column_names = [name[4:].replace('_', ' ') for name in all_getters]

In [ ]:
count = 0

start_time = time.time()
for column_name in column_names:
    count += 1
    if column_name not in ["segments pitches","segments timbre"] :
        print(count, column_name)
        save_to_csv(data[column_name], column_name.replace(' ', '_')+'.csv')
    
print("--- Execution time: %s seconds ---" % (time.time() - start_time))

In [7]:
base_directory

'/home/benjamin/./millionsongsubset_full/MillionSongSubset/data'

In [12]:
import tables

path  = base_directory+"/A/A/A/TRAAAAW128F429D538.h5"
content = open(path, "rb").read()

#h5 = hdf5_getters.open_h5_file_read()
#h5


def LoadData(content):
    h5 = tables.open_file("in-memory-sample.h5", driver="H5FD_CORE",
                              driver_core_image=content,
                              driver_core_backing_store=0)

    data = [get_year(h5),get_artist_terms(h5),get_artist_latitude(h5),get_artist_longitude(h5)]
    
    
    return data

def Filter(data):
        print(np.isnan(data[2]))
        return (data[0]!=0 and  not np.isnan(data[2]))
        
Filter(LoadData(content))

True


False

In [52]:
type(LoadData(content)[1][0])

numpy.bytes_

In [60]:
import json
class NumpyAwareJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray) and obj.ndim == 1:
            return obj.tolist()
        if isinstance(obj, np.int32):
            return int(obj)
        if isinstance(obj, np.bytes_) or isinstance(obj, bytes):
            return obj.decode("utf-8")
        print(type(obj))
        return json.JSONEncoder.default(self, obj)

def jsonData(data):
    return json.dumps(data,cls=NumpyAwareJSONEncoder)+',\n'

jsonData(LoadData(content))

'[0, ["hip hop", "underground rap", "g funk", "alternative rap", "gothic rock", "west coast rap", "rap", "club dance", "singer-songwriter", "chill-out", "underground hip hop", "rock", "gothic", "san francisco bay area", "indie", "american", "punk", "california", "industrial", "new york", "90s", "latin", "spanish", "dark", "ebm", "underground", "deathrock", "west coast", "san francisco", "producer", "oakland", "catalan", "barcelona", "doomsdope", "norcal", "west coast hip hop", "alternative rock"], NaN, NaN],\n'

In [62]:
import findspark
findspark.init("/home/benjamin/spark/spark-1.6.3-bin-hadoop2.6")
import pyspark 

conf = pyspark.SparkConf()
conf.setAppName("GET DATA")
conf.setMaster("yarn")
sc = pyspark.SparkContext(conf=conf)

sc.addPyFile("./hdf5_getters.py")

path='hdfs:/datasets/million-song_untar/A/A/A/*.h5'




sc.binaryFiles(path).map(lambda path,file : LoadData(file) ).filter(Filter).map(jsonData).saveAsTextFile("data.json")


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.io.EOFException: End of File Exception between local host is: "Benjamin-Surface/127.0.1.1"; destination host is: "iccluster060.iccluster.epfl.ch":8020; : java.io.EOFException; For more details see:  http://wiki.apache.org/hadoop/EOFException
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at org.apache.hadoop.net.NetUtils.wrapWithMessage(NetUtils.java:791)
	at org.apache.hadoop.net.NetUtils.wrapException(NetUtils.java:764)
	at org.apache.hadoop.ipc.Client.call(Client.java:1472)
	at org.apache.hadoop.ipc.Client.call(Client.java:1399)
	at org.apache.hadoop.ipc.ProtobufRpcEngine$Invoker.invoke(Pr